In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import os

In [2]:
path = 'D:/Programming/2020.11.AIcompet'

# https://systemtrade.tistory.com/345 << bs 사용법

In [3]:
driver = webdriver.Ie('D:/Programming/IEDriverServer2.exe')
time.sleep(1)
driver.get('https://www.scourt.go.kr/portal/information/finalruling/peruse/peruse_status.jsp')
time.sleep(1)

iframes = driver.find_elements_by_tag_name('iframe')
print('현재 페이지에 iframe은 %d개가 있습니다.' % len(iframes))
driver.switch_to.frame(iframes[0])

driver.find_element_by_xpath('''//input[@id='accept']''').click()
time.sleep(1)
driver.find_element_by_xpath('''//input[@id='name']''').send_keys('박동주')
time.sleep(1)
driver.find_element_by_xpath('''//input[@id='jumin1']''').send_keys('951030')
time.sleep(1)
driver.find_element_by_xpath('''//input[@id='jumin2']''').send_keys('1126827')
time.sleep(1)
driver.find_element_by_xpath('''//img[@id='auth']''').click()
time.sleep(1)
driver.find_element_by_xpath('''//a[@id='tb2']''').click()
time.sleep(1)
driver.find_element_by_xpath('''//input[@id='mq']''').send_keys('폭행 | 폭력')
time.sleep(1)
driver.find_element_by_xpath('''//input[@id='msearch-button']''').click()
time.sleep(1)
driver.find_element_by_xpath('''//select[@id='viewlist']''').click()
time.sleep(1)
driver.find_element_by_xpath('''//select[@id='viewlist']/option[@value='50']''').click()
time.sleep(1)
driver.find_element_by_xpath('''//dd[@class='btn']/img[@onclick='view()']''').click()
time.sleep(1)


soup = BeautifulSoup(driver.page_source, 'html.parser')
alist = []
blist = []

for n,i in enumerate(soup.find_all('td')):
    j = i.get_text()
    j = " ".join(j.split())
    alist.append(j)
    if ((n+1) % 6) == 0:
        blist.append(alist)
        alist = []

num = 2

while num < 83:
    
    path = '''//a[@onclick="javascript:mdetail_search('{}');"]'''.format(num)
    
    if driver.find_element_by_xpath(path):
        driver.find_element_by_xpath(path).click()
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        if soup.find('tr',{'class':'disable2'}):
            soup.find('tr',{'class':'disable2'}).decompose()

        for n,i in enumerate(soup.find_all('td')):
            j = i.get_text()
            j = " ".join(j.split())
            alist.append(j)
            if ((n+1) % 6) == 0:
                blist.append(alist)
                alist = []
        num = num + 1
        
    else:
        continue
    
df = pd.DataFrame(blist)
time.sleep(3)
print('완료')
driver.quit()

# for i in paging[2:13]:
#     i.click()
# driver.find_element_by_xpath('''//a[@onclick="javascript:mdetail_search('2');"]''').click()
# for page in paging[1:12]:
#     page.click()
#     time.sleep(1)

# paging = driver.find_elements_by_xpath('''//div[@id="paging"]/a[@href="javascript:"]''')
# while num < 10:
#     paging[num+1].click()
#     time.sleep(1)
#     soup = BeautifulSoup(driver.page_source, 'html.parser')
    
#     for n,i in enumerate(soup.find_all('td')):
#         j = i.get_text()
#         j = " ".join(j.split())
#         alist.append(j)
#         if ((n+1) % 6) == 0:
#             blist.append(alist)
#             alist = []
#     num = num + 2

현재 페이지에 iframe은 1개가 있습니다.
완료


In [4]:
df.rename(columns={0:'공백', 1:'선고일자',2:'법원명과 사건번호', 3:'사건명', 4:'주문', 5:'이유'}, inplace=True)
df.drop(columns='공백', inplace=True)

In [5]:
df

,선고일자,법원명과 사건번호,사건명,주문,이유
0,2020.10.15,대법원 [2019두40611],부당징계및부···[행정],"상고를 모두 기각한다. 상고비용 중 피고의 상고로 인한 부분은 피고가, 피고 보조참...","... 원고 F, G가 ‘순환전보 관련 사업소장 폭행’ 행위를 했다는 이유로 징계를..."
1,2020.10.14,청주지방법원 [2020가단25635],급여지급[민사],"피고는 원고에게 58,580,000원과 이에 대한 2019. 2. 14.부터 202...","... 학생들을 상대로 실시된 학교폭력(성폭력) 피해에 대한 설문조사 결과, ‘어깨..."
2,2020.10.14,제주지방법원 [2019나15489],손해배상(기···[민사],"1. 당심에서 확장된 원고의 청구에 따라 제1심판결의 주문 제1, 2항을 다음과 같...","... 한 사실이 인정되고, 이러한 사정이 피고의 폭행을 유발하였다고 보인다. 이와..."
3,2020.10.14,대구지방법원 김천지원 [2019가단34875],명의이전등록···[민사],"1. 피고는 원고에게, 가. 별지 목록 기재 자동차에 관하여 구미시 차량등록사업소 ...","... 항의하였고 이로 인하여 피고로부터 폭행을 당하였다고 진술하였으며, 이 사건...."
4,2020.10.08,수원지방법원 [2020구합61684],정직처분취소[행정],원고의 청구를 기각한다. 소송비용은 원고가 부담한다.,"..., 성범죄, 금품수수, 상해ㆍ폭행) 근절 추진계획의 일환으로 비위공무원에 대한..."
...,...,...,...,...,...
4090,2018.11.15,대전고등법원 [2018누11966],학교폭력 징···[행정],1. 피고의 항소를 기각한다. 2. 항소비용은 피고가 부담한다. 청구취지 및 항소취...,... 한다)은 이 사건 폭행에 관하여 다음과 같은 학교폭력사안조사 공동 보고서를....
4091,2018.11.15,수원지방법원 [2018나71588],손해배상(기···[민사],1. 항소를 기각한다. 2. 항소비용은 원고가 부담한다. 청구취지 및 항소취지 제1...,... 등 원고의 험담을 하고 손님들에게 폭행을 가하는 등 원고의 영업을 방해하였으...
4092,2018.11.15,전주지방법원 [2018나1974],손해배상(기···[민사],1. 피고의 항소를 기각한다. 2. 항소비용은 피고가 부담한다. 청구취지 및 항소취...,"... 붙게 되어 다투는 과정에서 피고가 원고를 폭행하게 된 것으로, 원고가 불법행..."
4093,2018.11.15,창원지방법원 [2018나1081],손해배상(기···[민사],1. 제1심판결의 원고 B에 대한 부분 중 아래에서 지급을 명하는 금액을 초과하는 ...,"... C, 피고인 D, 피고인 E의 폭력... 공동하여 피해자에게 폭행을 가하였다..."


In [6]:
df[480:520]

,선고일자,법원명과 사건번호,사건명,주문,이유
480,2020.06.11,서울행정법원 [2020구합58298],전학 처분 ···[행정],1. 피고가 2019. 12. 24. 원고에게 한 전학 처분을 취소한다. 2. 소송...,... ‘원고가 피해학생을 폭행함’을 원인으로 구 학교폭력예방법 제17조에 따라.....
481,2020.06.11,서울행정법원 [2020구단7187],난민불인정결···[행정],1. 원고의 청구를 기각한다. 2. 소송비용은 원고가 부담한다.,... 대부분 중간급 또는 고위급 이상의 지도자들과 폭력적인 성향이 짙어진 반정부시...
482,2020.06.11,서울중앙지방법원 [2020가합510343],혼인관계 부···[민사],1.원고와 피고 사이에 1992.3.5.부터 2004.2.5.까지 실질적인 혼인관계...,1. 청구의 표시 : 별지 청구원인 기재와 같다. 2. 무변론 판결(민사소송법 제2...
483,2020.06.11,서울행정법원 [2019구합74935],재심판정 취···[행정],1. 원고의 청구를 기각한다. 2. 소송비용은 보조참가로 인한 부분을 포함하여 모두...,재심판정의 경위 등 가. 피고보조참가인(이하 ‘참가인 회사’)은 1970. 1. 5...
484,2020.06.11,수원지방법원 [2019구합70170],원장 자격정···[행정],1. 원고의 청구를 기각한다. 2. 소송비용은 원고가 부담한다.,"... 볼 자료는 없다. 원고에게 아동을 폭행하거나 학대할 의도는 보이지 않으나,...."
485,2020.06.11,서울고등법원 [2019누64909],부당해고구제···[행정],1. 원고의 항소를 기각한다. 2. 항소비용은 보조참가로 인한 부분을 포함하여 원고...,"... 출근한 첫 날인 2018. 1. 29. 동료 직원을 모욕, 폭행하였고, 참가..."
486,2020.06.11,대전고등법원 [2019누12911],평균임금정정···[행정],1. 원고의 항소를 기각한다. 2. 항소비용은 원고가 부담한다. 청구취지 및 항소취...,... 이 사건 회사를 퇴사하기 전까지 특별한 폭력적인 성향이나 이 사건 상병과 같...
487,2020.06.11,대전고등법원 [2019누12843],해임처분취소[행정],1. 원고의 항소를 기각한다. 2. 항소비용은 원고가 부담한다. 청구취지 및 항소취...,"...폭행모욕언어폭력),... 같은 모욕, 언어폭력 행위를 하였던 것으로... 더보..."
488,2020.06.11,수원지방법원 [2019나81346],손해배상(기···[민사],"제1심판결 중 아래에서 지급을 명하는 금액을 초과하는 피고들 패소 부분을 취소하고,...",... 이 사건 상해로 인하여 폭력... 2018. 12. 26. 피고들에 대한 폭...
489,2020.06.11,서울중앙지방법원 [2019나66547],손해배상(기···[민사],1. 원고의 항소를 모두 기각한다. 2. 항소비용은 원고가 부담한다. 청구취지 및 ...,".... 당시 원고는 F, G, H이 공동하여 원고를 폭행하였다고 진술하였다. 나...."


In [14]:
tq = pd.read_csv("D:/Programming/2020.11.AIcompet/law_crawling11.csv", sep=",", encoding="CP949")
tq

,Unnamed: 0,선고일자,법원명과 사건번호,사건명,주문,이유
0,0,2020.10.15,대법원 [2019두40611],부당징계및부···[행정],"상고를 모두 기각한다. 상고비용 중 피고의 상고로 인한 부분은 피고가, 피고 보조참...","... 원고 F, G가 ‘순환전보 관련 사업소장 폭행’ 행위를 했다는 이유로 징계를..."
1,1,2020.10.14,청주지방법원 [2020가단25635],급여지급[민사],"피고는 원고에게 58,580,000원과 이에 대한 2019. 2. 14.부터 202...","... 학생들을 상대로 실시된 학교폭력(성폭력) 피해에 대한 설문조사 결과, ‘어깨..."
2,2,2020.10.14,제주지방법원 [2019나15489],손해배상(기···[민사],"1. 당심에서 확장된 원고의 청구에 따라 제1심판결의 주문 제1, 2항을 다음과 같...","... 한 사실이 인정되고, 이러한 사정이 피고의 폭행을 유발하였다고 보인다. 이와..."
3,3,2020.10.14,대구지방법원 김천지원 [2019가단34875],명의이전등록···[민사],"1. 피고는 원고에게, 가. 별지 목록 기재 자동차에 관하여 구미시 차량등록사업소 ...","... 항의하였고 이로 인하여 피고로부터 폭행을 당하였다고 진술하였으며, 이 사건...."
4,4,2020.10.08,수원지방법원 [2020구합61684],정직처분취소[행정],원고의 청구를 기각한다. 소송비용은 원고가 부담한다.,"..., 성범죄, 금품수수, 상해ㆍ폭행) 근절 추진계획의 일환으로 비위공무원에 대한..."
...,...,...,...,...,...,...
4090,4090,2018.11.15,대전고등법원 [2018누11966],학교폭력 징···[행정],1. 피고의 항소를 기각한다. 2. 항소비용은 피고가 부담한다. 청구취지 및 항소취...,... 한다)은 이 사건 폭행에 관하여 다음과 같은 학교폭력사안조사 공동 보고서를....
4091,4091,2018.11.15,수원지방법원 [2018나71588],손해배상(기···[민사],1. 항소를 기각한다. 2. 항소비용은 원고가 부담한다. 청구취지 및 항소취지 제1...,... 등 원고의 험담을 하고 손님들에게 폭행을 가하는 등 원고의 영업을 방해하였으...
4092,4092,2018.11.15,전주지방법원 [2018나1974],손해배상(기···[민사],1. 피고의 항소를 기각한다. 2. 항소비용은 피고가 부담한다. 청구취지 및 항소취...,"... 붙게 되어 다투는 과정에서 피고가 원고를 폭행하게 된 것으로, 원고가 불법행..."
4093,4093,2018.11.15,창원지방법원 [2018나1081],손해배상(기···[민사],1. 제1심판결의 원고 B에 대한 부분 중 아래에서 지급을 명하는 금액을 초과하는 ...,"... C, 피고인 D, 피고인 E의 폭력... 공동하여 피해자에게 폭행을 가하였다..."
